# Generative AI Plot / Choice Generation Demo + Script

In [27]:
!pip install together==1.2.0
!pip install wikipedia-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=f425365d20b52909a7d6d38ae4c5875aae0256a8b207c2dc65dfc17d946f7835
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [10]:
import wikipediaapi
import json

## Proof Of Concept

In [13]:
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')


def get_plot(title):
    """
    Retrieves the plot summary of the given title from Wikipedia.
    """
    page = wiki_wiki.page(title)
    if not page.exists():
        print(f"Page '{title}' does not exist.")
        return None

    possible_sections = ['Plot', 'Synopsis', 'Summary', 'Plot summary', 'Story']
    plot_section = None
    for section_title in possible_sections:
        plot_section = page.section_by_title(section_title)
        if plot_section:
            break
    if plot_section:
        return plot_section.text
    else:
        print(f"No plot section found for '{title}'.")
        return None

In [22]:
IP = 'Dracula'

system_prompt = f"""
Your job is to adapt the story of {IP} and create a branching storyline that incorporates similar elements while introducing new twists and engaging paths for players.
Instructions:
- Only generate in plain text without formatting.
- Use simple, clear language that is easy to understand and avoids being overly descriptive.
- Ensure each branching storyline offers meaningful player choices and consequences.
- Stay concise, limiting each storyline description to 3-5 sentences.
"""

plot_prompt = f"""
Generate a creative, original adaptation of {IP}'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.

Output content in the form:
Plot Summary: <PLOT SUMMARY>
Branching Storylines: <BRANCHING STORYLINE DESCRIPTIONS>

{IP} Plot Summary: {get_plot(IP)}"""


In [23]:
plot_prompt

'\nGenerate a creative, original adaptation of Dracula\'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.\n\nOutput content in the form:\nPlot Summary: <PLOT SUMMARY>\nBranching Storylines: <BRANCHING STORYLINE DESCRIPTIONS>\n\nDracula Plot Summary: Jonathan Harker, a newly qualified English solicitor, visits Count Dracula at his castle in the Carpathian Mountains to help the Count purchase a house near London. Ignoring the Count\'s warning, Harker wanders the castle at night and encounters three vampire women; Dracula rescues Harker, and gives the women a small child bound inside a bag. Harker awakens in bed; soon after, Dracula leaves the castle, abandoning him to the women. Harker escapes and ends up delirious in a Budapest hospital. Dracula takes a ship called the Demeter for England with boxes of earth from his castle. The captain\'s log narrates the crew\'s disappearance until he alone remains, bound to the helm to ma

In [24]:
from together import Together

client = Together(api_key='0256f61cf32560dba2fb9c36573f46b473dbd1f2f0d95a731eeaf01b458e7bed')

output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": plot_prompt}
    ],
)

In [25]:
plot_output =output.choices[0].message.content
print(plot_output)

Here is a creative, original adaptation of Dracula's plot with 4 new branching storylines for players:

Plot Summary: In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with facilitating the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late.

Branching Storylines:

**Path 1: The Mysterious Heir**
Emily discovers that she is the last living heir of the original owner of the mansion, and that the Count's true intention is to claim the estate's dark legacy for himself. Players must navigate the complex web of family secrets and ancient curses to uncover the truth about their own past and the source of the Count's power.

**Path 2: The Vampire's Prey

In [96]:
def generate_continuation(context, previous_scenario=None, previous_choice=None, num_choices=4, choices_left=10):
    """
    Generates a continuation of the story based on the given context and an optional choice.
    choices_left details the amount of urgency left, to account for story pacing.
    """
    system_prompt = f"""
    Your job is to tell a continuation of the story based on what previously happened.
    You will be given the overall plot synopsis to help guide the story in that direction.
    The player has {choices_left} choice(s) left.

    Instructions:
    - Use simple, clear language that is easy to understand and avoids being overly descriptive.
    - Stay in third-person point of view.
    - Ensure each branching storyline offers meaningful player choices and consequences.
    - Stay concise, limiting each storyline description to 3-5 sentences.
    - The player has around 10 choices in total. If they have 5-7 left, ramp up the story, aiming for 3-4 choices left as the climax. Begin to aim for an interesting conclusion at 2-1 choices left.
    - All outputs should be in JSON format like so. Do not include anything else:
    {{
      "story_continuation": "[Your story continuation here]",
      "choices": [
        {{
          "choice": 1,
          "action": "[First possible action]"
        }},
        {{
          "choice": 2,
          "action": "[Second possible action]"
        }}
        ...
      ]
    }}
    """

    if previous_choice:
        prompt = f"""
        Write a continuation of the story that progresses the story forward. Include {num_choices} choices for the player to choose from given this generated situation.
        All outputs should be in JSON format. Do not include anything else.

        The following scenario has just occured:
        {previous_scenario}

        The player made the following choice:
        {previous_choice}

        Plot Summary: {context}
        """
    else:
        prompt = f"""
        We are at the start of our adventure. Write a continuation of the story that progresses the story forward. Include {num_choices} choices for the player to choose from given this generated situation.
        All outputs should be in JSON format.

        Plot Summary: {context}
        """

    output = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": prompt}
      ],
    )
    return output.choices[0].message.content

In [57]:
# Example:
# Generated Plot Summary now becomes context: In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with facilitating the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late.
# Each path is added to the context (can also put the path name on screen, which I think is cool and engaging): e.g. Emily discovers that she is the last living heir of the original owner of the mansion, and that the Count's true intention is to claim the estate's dark legacy for himself. Players must navigate the complex web of family secrets and ancient curses to uncover the truth about their own past and the source of the Count's power.

In [58]:
example_context = "In the small town of Ravenswood, a mysterious stranger named Count Draconis arrives, seeking to purchase an old mansion on the outskirts of town. Local solicitor, Emily Welles, is tasked with facilitating the sale, but soon discovers that the Count's true intentions are far more sinister. As the town falls under the Count's dark influence, Emily must join forces with a group of allies to uncover the truth behind the Count's powers and stop him before it's too late. Emily discovers that she is the last living heir of the original owner of the mansion, and that the Count's true intention is to claim the estate's dark legacy for himself. Players must navigate the complex web of family secrets and ancient curses to uncover the truth about their own past and the source of the Count's power."

output = generate_continuation(example_context, choices_left=10)
output

'{\n  "story_continuation": "As Emily delves deeper into the mystery, she begins to experience strange and unsettling occurrences. Doors slam shut on their own, and she starts to feel an eerie presence lurking in the shadows. One night, while researching in the town\'s archives, she stumbles upon an ancient diary belonging to her ancestor, the original owner of the mansion. The diary speaks of a dark family secret and a powerful artifact hidden within the mansion\'s walls. Suddenly, the lights in the archive room begin to flicker, and Emily feels an intense chill run down her spine. She realizes she\'s not alone and must make a decision quickly.",\n  "choices": [\n    {\n      "choice": 1,\n      "action": "Investigate the strange occurrences in the archive room further, trying to uncover the source of the disturbance."\n    },\n    {\n      "choice": 2,\n      "action": "Leave the archive room immediately and head back to her office to gather her thoughts and plan her next move."\n   

In [59]:
parsed_data = json.loads(output)
cleaned_json = json.dumps(parsed_data, indent=4)
print(cleaned_json)

{
    "story_continuation": "As Emily delves deeper into the mystery, she begins to experience strange and unsettling occurrences. Doors slam shut on their own, and she starts to feel an eerie presence lurking in the shadows. One night, while researching in the town's archives, she stumbles upon an ancient diary belonging to her ancestor, the original owner of the mansion. The diary speaks of a dark family secret and a powerful artifact hidden within the mansion's walls. Suddenly, the lights in the archive room begin to flicker, and Emily feels an intense chill run down her spine. She realizes she's not alone and must make a decision quickly.",
    "choices": [
        {
            "choice": 1,
            "action": "Investigate the strange occurrences in the archive room further, trying to uncover the source of the disturbance."
        },
        {
            "choice": 2,
            "action": "Leave the archive room immediately and head back to her office to gather her thoughts an

In [67]:
output = generate_continuation(example_context,
                               previous_scenario=parsed_data.get("story_continuation"),
                               previous_choice=parsed_data.get("choices")[0]['action'],
                               choices_left=9)
output

'{\n  "story_continuation": "As Emily investigates the strange occurrences in the archive room, she notices that the flickering lights seem to be emanating from a specific section of the shelves. She approaches the area cautiously, her heart racing with anticipation. Suddenly, a book falls off the shelf, opening to a page with a cryptic message scrawled in the margin. The message reads: \'Beware the eyes that watch from the shadows.\' Emily feels a shiver run down her spine as she realizes that she is being watched. She looks around the room, but sees no one. The air is thick with tension, and she knows she must act quickly to uncover the truth.",\n  "choices": [\n    {\n      "choice": 1,\n      "action": "Search the archive room for any hidden cameras or surveillance devices."\n    },\n    {\n      "choice": 2,\n      "action": "Try to decipher the cryptic message and its connection to the dark family secret."\n    },\n    {\n      "choice": 3,\n      "action": "Leave the archive roo

In [68]:
parsed_data = json.loads(output)
cleaned_json = json.dumps(parsed_data, indent=4)
print(cleaned_json)

{
    "story_continuation": "As Emily investigates the strange occurrences in the archive room, she notices that the flickering lights seem to be emanating from a specific section of the shelves. She approaches the area cautiously, her heart racing with anticipation. Suddenly, a book falls off the shelf, opening to a page with a cryptic message scrawled in the margin. The message reads: 'Beware the eyes that watch from the shadows.' Emily feels a shiver run down her spine as she realizes that she is being watched. She looks around the room, but sees no one. The air is thick with tension, and she knows she must act quickly to uncover the truth.",
    "choices": [
        {
            "choice": 1,
            "action": "Search the archive room for any hidden cameras or surveillance devices."
        },
        {
            "choice": 2,
            "action": "Try to decipher the cryptic message and its connection to the dark family secret."
        },
        {
            "choice": 3,


## Recursive Script

In [97]:
all_jsons = []
GENERATION_THRESHOLD = 8

def initialize_story(IP):
    system_prompt = f"""
    Your job is to adapt the story of {IP} and create a branching storyline that incorporates similar elements while introducing new twists and engaging paths for players.
    Instructions:
    - Use simple, clear language that is easy to understand and avoids being overly descriptive.
    - Ensure each branching storyline offers meaningful player choices and consequences.
    - Stay concise, limiting each storyline description to 3-5 sentences.
    - All outputs should be in JSON format like so. Do not include anything else:
    {{
      "plot_summary": "[Your initial plot summary here]",
      "branching_storylines": [
        {{
          "path": 1,
          "path_name": "Title of path"
          "story_line": "[Summary of first storyline]"
        }},
        {{
          "path": 2,
          "path_name": "Title of path"
          "story_line": "[Summary of second storyline]"
        }}
        ...
      ]
    }}
    """

    plot_prompt = f"""
    Generate a creative, original adaptation of {IP}'s plot, introducing similar themes and elements while creating 4 new, engaging branching storylines for players.
    All outputs should be in JSON format. Do not include anything else.

    {IP} Plot Summary: {get_plot(IP)}"""

    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": plot_prompt}
        ],
    )

    return output.choices[0].message.content

# potential recursive method:
def populate_JSON(context, previous_scenario, previous_choice, num_choices=4, choices_left=10):
    if choices_left <= GENERATION_THRESHOLD:
      return

    output = generate_continuation(context, previous_scenario, previous_choice, num_choices, choices_left)

    # store generated continuation (can revise)
    parsed_data = json.loads(output)
    all_jsons.append(parsed_data)

    current_scenario = parsed_data.get("story_continuation")
    for choice in parsed_data.get("choices"):
        current_choice = choice["action"]
        populate_JSON(context,
                      previous_scenario=current_scenario,
                      previous_choice=current_choice,
                      num_choices=num_choices,
                      choices_left=choices_left-1)

def create_story(IP, num_choices=4, choices_left=10):
    story = json.loads(initialize_story(IP))
    initial_context = story.get("plot_summary")
    for branch in story.get("branching_storylines"):
        path_name = branch["path_name"] # TODO: add to final JSON + Unity
        story_line = branch["story_line"]

        # context for each branch is initial context + overall plot of specific branch
        context = initial_context + " " + story_line
        populate_JSON(context, previous_scenario=None, previous_choice=None, num_choices=num_choices, choices_left=choices_left)


In [99]:
create_story('Dracula')

In [100]:
story

{'plot_summary': "In the modern city of New Haven, a mysterious corporation called 'Eclipse' has taken over the city's infrastructure, and people are disappearing at an alarming rate. The main character, a brilliant journalist named Maya, receives a cryptic message from an anonymous source claiming to have information about Eclipse's true intentions. As Maya delves deeper into the mystery, she discovers that Eclipse is led by a charismatic and enigmatic figure known only as 'The Archon', who seems to have supernatural abilities.",
 'branching_storylines': [{'path': 1,
   'path_name': "The Whistleblower's Trail",
   'story_line': "Maya follows the trail of clues left by the anonymous source, leading her to a hidden underground bunker where she finds evidence of Eclipse's sinister experiments. She must decide whether to go public with the information or continue investigating to uncover more secrets."},
  {'path': 2,
   'path_name': "The Vampire's Prey",
   'story_line': 'Maya is attacke

In [101]:
all_jsons

[{'story_continuation': "Maya's heart raced as she explored the hidden bunker, uncovering disturbing evidence of Eclipse's experiments. She found records of human subjects being subjected to strange procedures, and cryptic notes referencing 'The Ascension Project'. Suddenly, she heard footsteps echoing through the corridors, and realized she wasn't alone. Maya knew she had to act fast, or risk being caught by Eclipse's agents.",
  'choices': [{'choice': 1,
    'action': 'Maya quickly gathers as much evidence as she can and makes a run for the exit, hoping to escape undetected.'},
   {'choice': 2,
    'action': 'Maya decides to investigate the source of the footsteps, trying to gather more information about who or what is coming for her.'},
   {'choice': 3,
    'action': 'Maya searches the bunker for a way to destroy the evidence, hoping to prevent Eclipse from continuing their sinister experiments.'},
   {'choice': 4,
    'action': 'Maya tries to find a way to contact the anonymous sou